In [10]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
train_df = pd.read_csv("dsjtzs_txfz_training.txt",header=None,delim_whitespace=True)
train_df.columns = ["id","points_str","target_point","label"]


train_df.set_index(["id"],inplace=True)

def f(s):
    s =str(s)
    return s[0:-1]
train_df["points_str"]= train_df["points_str"].apply(f)


def  point_sort(x,y):
    if (x[2]>y[2]):
        return 1
    elif (x[2]<y[2]):
        return -1
    else:
        return 0
def f(t):
    s=str(t["points_str"])
    splited=s.split(";")
    ret = []
    for item in splited:
        item_splited = item.split(",")
        item_tuple = ( int(item_splited[0]),  int(item_splited[1]) ,int(item_splited[2]) )
        ret.append(item_tuple)
    sorted_arr = sorted(ret,point_sort)
    return sorted_arr
train_df["points_sorted_list"]=train_df[["points_str"]].apply(f,axis=1)

def f(s):
    sorted_arr=list(s["points_sorted_list"])
    _sum_x =0 
    _sum_y=0 
    _count =0
    ret=[]
    for i in range(0,len(sorted_arr)-1):
        if (sorted_arr[i][2]!=sorted_arr[i+1][2]):
                if _count==0:
                    ret.append(sorted_arr[i])
                else:
                    _count +=1
                    _sum_x+=sorted_arr[i][0]
                    _sum_y+=sorted_arr[i][1]
                    average_x =float( _sum_x ) / _count
                    average_y =float(_sum_y)/_count
                    ret.append((average_x,average_y,sorted_arr[i][2]))
                    _count =0
                    _sum_x=0
                    _sum_y=0
        else: 
            _sum_x += sorted_arr[i][0]
            _sum_y+=sorted_arr[i][1]
            _count+=1
    
    index = len(sorted_arr)-1
    if (sorted_arr[index][2] == sorted_arr[index-1][2]):
        _count+=1
        _sum_x+=sorted_arr[index][0]
        _sum_y+=sorted_arr[index][1]
        average_x =float( _sum_x ) / _count
        average_y =float(_sum_y)/_count
        ret.append((average_x,average_y,sorted_arr[index][2]))
    else:
        ret.append(sorted_arr[index])
    return ret

train_df["points_diff_time_list"]=train_df[["points_sorted_list"]].apply(f,axis=1)

# print train_df.loc[1,"points_str"]
# print "-----"
# print train_df.loc[1,"points_sorted_list"]

# points 都有数据
#print pd.isnull(train_df["points"]).value_counts()

# 2600 个1 正例；400个0 负例 (id:2601-3000)
#train_df.label.value_counts()

# id 都是不同的
#train_df.id.value_counts().shape[0]

In [11]:
def f_x(s):
    s =list(s)
    ret =[]
    for item in s:
        ret.append(int(item[0]))
    return ret
        
train_df["points_x"]= train_df["points_diff_time_list"].apply(f_x)

def f_y (s):
    s=list(s)
    ret=[]
    for item in s:
        ret.append(int(item[1]))
    return ret
train_df["points_y"]=train_df["points_diff_time_list"].apply(f_y)

def f_time (s):
    s =list(s)
    ret=[]
    for item in s:
        ret.append(int(item[2]) )
    return ret
train_df["points_time"]=train_df["points_diff_time_list"].apply(f_time)

def f_target_point_x(s):
    s = str(s)
    splited = s.split(",")
    return float(splited[0])
train_df["target_point_x"]=train_df["target_point"].apply(f_target_point_x)

def f_target_point_y (s):
    s =str(s)
    splited =s.split(",")
    return float(splited[1])
train_df["target_point_y"]=train_df["target_point"].apply(f_target_point_y)



train_df =train_df[["points_x","points_y","points_time","target_point_x","target_point_y","target_point","label"]]

print train_df.head(10)



                                             points_x  \
id                                                      
1   [353, 367, 388, 416, 500, 584, 675, 724, 780, ...   
2   [283, 290, 297, 304, 311, 325, 339, 346, 360, ...   
3   [255, 297, 353, 423, 479, 542, 605, 668, 745, ...   
4   [437, 549, 605, 661, 731, 780, 822, 871, 885, ...   
5   [248, 255, 297, 339, 451, 577, 717, 885, 948, ...   
6   [171, 178, 178, 206, 353, 486, 668, 815, 899, ...   
7   [227, 234, 241, 248, 255, 262, 269, 276, 283, ...   
8   [199, 241, 255, 283, 339, 395, 465, 493, 507, ...   
9   [178, 192, 199, 206, 220, 227, 241, 255, 262, ...   
10  [290, 318, 346, 374, 437, 493, 570, 675, 766, ...   

                                             points_y  \
id                                                      
1   [2607, 2607, 2620, 2620, 2620, 2620, 2620, 262...   
2   [2490, 2490, 2490, 2490, 2490, 2503, 2503, 250...   
3   [2503, 2503, 2529, 2542, 2555, 2568, 2581, 260...   
4   [2724, 2711, 2724, 2750, 2

In [3]:
# plt.figure(figsize=(8,7),dpi=98)
# p1 = plt.subplot(211)
# p2 = plt.subplot(212)
# x = train_df.loc[1,"points_x"]
# y  = train_df.loc[1,"points_y"]
# target_x = train_df.loc[1,"target_point_x"]
# target_y = train_df.loc[1,"target_point_y"]
# print x
# print y
# p1.plot(x,y,'-',color='r',label="person")
# p1.plot(target_x,target_y,marker='*')
# p1.legend()
# plt.show()
ids = [1,2,3,4,2601,2602,2603,2604]
for id in ids:
    plt.figure(id)  #一个Figure对象可以包含多个子图（Axes）．指定了plt.figure(i)之后的操作就是在图表i 下进行的
    x = train_df.loc[id,"points_x"]
    y  = train_df.loc[id,"points_y"]
    target_x = train_df.loc[id,"target_point_x"]
    target_y = train_df.loc[id,"target_point_y"]
#     print x
#     print y
    print id
    print (target_x,target_y)
    print "-----------"
    if train_df.loc[id,"label"]==1:
        labelStr="person"
    else:
        labelStr="robort"
    plt.scatter(x,y,c="r",marker="*")
#  plt.plot(x,y,'-',color='r',label=labelStr)
 #  plt.plot(target_x,target_y,marker='*')
    plt.grid(True)
    plt.legend()
plt.show()

1
(1420.5, 202.0)
-----------
2
(1018.0, 189.0)
-----------
3
(601.5, 559.5)
-----------


/usr/lib/pymodules/python2.7/matplotlib/axes.py:4747: UserWarning: No labeled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labeled objects found. "


4
(675.0, 202.0)
-----------
2601
(1470.5, 1144.5)
-----------
2602
(1329.5, 722.0)
-----------
2603
(1227.5, 306.0)
-----------
2604
(1263.0, 215.0)
-----------


/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [22]:
def  f (s):
    ret = []
    points_x= list(s["points_x"])
    points_time =list(s["points_time"])
    for x in range(1,len(points_x)):
        velocity = (float(points_x[x])-points_x[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_x"]= train_df[["points_x","points_time"]].apply(f,axis=1)


def f(s):
    ret =[]
    points_y = list(s["points_y"])
    points_time = list(s["points_time"])
    for x in range(1,len(points_y)):
        velocity = (float(points_y[x])-points_y[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_y"]=train_df[["points_y","points_time"]].apply(f,axis=1)

def f(s):
    ret =[]
    velocity_x =list(s["velocity_x"])
    times = list(s["points_time"])
    if len(velocity_x)>1:
        for x in range (1,len (velocity_x)):
            accelerate_x = (float(velocity_x[x])- velocity_x[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_x)
    return ret
train_df["accelerate_x"]=train_df[["velocity_x","points_time"]].apply(f,axis=1)


def f(s):
    ret = []
    velocity_y =list(s["velocity_y"])
    times=list(s["points_time"])
    if (len(velocity_y)>1):
        for x in range(1,len(velocity_y)):
            accelerate_y = (float(velocity_y[x])-velocity_y[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_y)
    return ret
train_df["accelerate_y"]=train_df[["velocity_y","points_time"]].apply(f,axis=1)



In [23]:
import numpy as np
list=[1,2,3,13]
l=np.array(list)
